In [ ]:
import operator
import warnings
from typing import Annotated, TypedDict

import torch
from dotenv import load_dotenv
from IPython.display import Image
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_openai import ChatOpenAI
from transformers import logging

from medixar.agent import *
from medixar.tools import *
from medixar.utils import *

warnings.filterwarnings("ignore")
logging.set_verbosity_error()
_ = load_dotenv()

PROMPT_FILE = "medixar/docs/system_prompts.txt"

"""
1. Change the LLM to public LLM
2. Add other tools + integrate tool baackend
4. Choose agent name
5. Explore code generation
6. SFT or example database, prompt enhancing to make tool call better
"""

In [ ]:
report_tool = RadiologyReportGeneratorTool()
organ_size_tool = OrganSizeMeasurementTool()
chest_xray_tool = ChestXRayClassifierTool()

print(f"Type: {type(report_tool)}, Name: {report_tool.name}")
print(f"Type: {type(organ_size_tool)}, Name: {organ_size_tool.name}")
print(f"Type: {type(chest_xray_tool)}, Name: {chest_xray_tool.name}")

In [ ]:
prompts = load_prompts_from_file(PROMPT_FILE)
prompt = prompts["MEDICAL_ASSISTANT"]
print(prompt)

In [ ]:
checkpointer = MemorySaver()
model = ChatOpenAI(model="gpt-4o-mini", temperature=0, top_p=0.95)
agent = Agent(model, [report_tool, organ_size_tool, chest_xray_tool], system=prompt, checkpointer=checkpointer)
thread = {"configurable": {"thread_id": "1"}}

Image(agent.graph.get_graph().draw_png())

In [23]:
messages = [HumanMessage(content="Does the image `demo/chest/pneumonia4.jpg` show pneumonia?")]
messages = [HumanMessage(content="Describe the radiology image in detail.")]
messages = [HumanMessage(content="What is the volume of the heart (mL)?")]
messages = [HumanMessage(content="What was the first question I asked?")]
messages = [HumanMessage(content="What is the name and size of first organ mention in radiology report of `image.png`")]

In [ ]:
async for event in agent.graph.astream_events({"messages": messages}, thread, version="v1"):
        kind = event["event"]
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                print(content, end="")